<a href="https://colab.research.google.com/github/gowthambalachandhiran/AUKBC-LLM-Toolkit/blob/main/notebook/FineTuneUnsloth/Gpt_OSS_fine_tuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip uninstall numpy -y
!pip install numpy==2.0.2

# Install Unsloth and dependencies
%%capture
!pip install --upgrade -qqq uv
try:
    import numpy
    get_numpy = f"numpy=={numpy.__version__}"
except:
    get_numpy = "numpy"

!uv pip install -qqq \
    "torch>=2.8.0" \
    "triton>=3.4.0" \
    {get_numpy} \
    torchvision \
    bitsandbytes \
    "transformers>=4.55.3" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth"

!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers
!uv pip install --no-deps trl==0.22.2

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 40.7 MB/s eta 0:00:00


UsageError: Line magic function `%%capture` not found.


## Installing depedencies

In [ ]:
!pip install unsloth

## We will be using unsloth for fine tuning the model. I am using free version of Colab Testla T4. It is exceptionally easier to fine tune the models . We use Unsloth for following reasons

- Highly compatible with Hugging face libraries such as llama,Phi,mistrel etc
- Easier code base . Very tight in coding.
- Replace Pytorch kernel with Triton fused kernels which is much more faster than Pytorch kernels and can level the speed 3x for fine tuning
- Active engagement of community in case of issues.
- Most of the traning is handled by kernel with very little handover to python
- Supports 4 bit quantization to support extreme light weight models.

In [2]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
import transformers

# Configuration
max_seq_length = 1024
dtype = None  # Auto-detect
load_in_4bit = True  # Use 4-bit quantization for memory efficiency

print("Loading GPT-OSS model with Unsloth optimizations...")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
Loading GPT-OSS model with Unsloth optimizations...


# We take up the gpt-oss-20b => 20 billion model on free gpu offered by colab . The experiement cant get better than this

In [3]:
# Load the model with Unsloth's optimizations
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b",  # 20B parameter model
    dtype=dtype,
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
    full_finetuning=False,
)

print("Model loaded successfully!")
print(f"Max memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

==((====))==  Unsloth 2025.10.7: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Model loaded successfully!
Max memory: 15.83 GB


# We only Select very few layers for fine tuning. Less than 2 % of the whole model yet having a very significant results in inference

In [4]:
# Configure LoRA (Low-Rank Adaptation) for efficient fine-tuning


model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # LoRA rank (higher = more parameters, but more memory)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized checkpointing
    random_state=3407,
    use_rslora=False,
)


print("PEFT model configured with LoRA adapters")

Unsloth: Making `model.base_model.model.model` require gradients
PEFT model configured with LoRA adapters


# I took the mental health data set . This is more conversational chat between client and psychiatrist. I want to experiment with oss model to see how it responds between base version and fine tuned version

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset

# Login to Hugging Face (required for dataset access)
notebook_login()

# Load the psychology counseling dataset
ds = load_dataset("Amod/mental_health_counseling_conversations")

print(f"Dataset loaded: {len(ds['train'])} conversations")
print(f"\n Sample conversation:")
print(f"Context: {ds['train'][0]['Context'][:200]}...")
print(f"Response: {ds['train'][0]['Response'][:200]}...")

## Formating the input interms of system, user and response configuration

In [6]:
def formatting_prompts_func(examples):
    """
    Format the dataset into chat templates for the model.
    Each conversation includes:
    - System message: Sets the role as a psychologist
    - User message: The client's context/question
    - Assistant message: The therapist's response
    """
    convos = []

    for i in range(len(examples["Context"])):
        formatted_messages = [
            {
                "role": "system",
                "content": "You are a helpful and empathetic psychologist."
            },
            {
                "role": "user",
                "content": examples["Context"][i]
            },
            {
                "role": "assistant",
                "content": examples["Response"][i]
            }
        ]
        convos.append(formatted_messages)

    # Apply chat template
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        )
        for convo in convos
    ]

    return {"text": texts}

## Sample from Data Set

In [7]:
# Process the dataset
processed_ds = ds.map(
    formatting_prompts_func,
    batched=True,
    desc="Formatting conversations"
)

print("Dataset formatted successfully!")
print(f"\nSample formatted text:")
print(processed_ds['train'][0]['text'][:500] + "...")

Formatting conversations:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset formatted successfully!

Sample formatted text:
<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-10-22

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

You are a helpful and empathetic psychologist.<|end|><|start|>user<|message|>I'm going through some things with my feelings and mysel...


In [8]:
processed_ds['train'][0]['text']

"<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-10-22\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a helpful and empathetic psychologist.<|end|><|start|>user<|message|>I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?<|end|><|start|>assistant<|message|>If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem

In [9]:
def tokenize_function(examples):
    """
    Tokenize the formatted text for model training.
    """
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_seq_length,
        padding=False,  # Dynamic padding in data collator
        return_tensors=None,
    )

    # For causal language modeling, labels are the same as input_ids
    outputs["labels"] = outputs["input_ids"].copy()

    return outputs

# Tokenize the dataset
tokenized_ds = processed_ds.map(
    tokenize_function,
    batched=True,
    remove_columns=processed_ds['train'].column_names,
    desc="Tokenizing dataset"
)

print("Dataset tokenized!")
print(f"Sample tokenized length: {len(tokenized_ds['train'][0]['input_ids'])} tokens")

Tokenizing dataset:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset tokenized!
Sample tokenized length: 357 tokens


In [10]:
# IMPORTANT: Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    print("⚠️  Set pad_token to eos_token")

In [11]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Configure training parameters
training_args = TrainingArguments(
    output_dir="./gpt-oss-psychologist",
    num_train_epochs=1,
    per_device_train_batch_size=2,  # Adjust based on GPU memory
    gradient_accumulation_steps=2,  # Effective batch size = 2 * 4 = 8
    learning_rate=2e-4,
    fp16=True,  # Mixed precision training
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    report_to="none",  # Disable wandb
    remove_unused_columns=False,
)

# Initialize trainer
# Use DataCollatorForSeq2Seq for proper padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,  # Pad sequences to the longest in the batch
    max_length=max_seq_length,
    label_pad_token_id=-100,  # Ignore padding tokens in loss
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    data_collator=data_collator,
)

Let's break down the `TrainingArguments` configuration:

- **`output_dir="./gpt-oss-psychologist"`**: This specifies the directory where the training checkpoints and outputs will be saved.
- **`num_train_epochs=1`**: This sets the number of times the trainer will iterate over the entire training dataset. In this case, it's set to 1 epoch.
- **`per_device_train_batch_size=2`**: This determines the batch size used for training on each GPU. A smaller batch size is used here to manage GPU memory.
- **`gradient_accumulation_steps=2`**: This setting accumulates gradients over several batches before performing a backward pass and updating the model weights. This effectively increases the batch size without requiring more memory. The effective batch size is `per_device_train_batch_size * gradient_accumulation_steps = 2 * 2 = 4`.
- **`learning_rate=2e-4`**: This is the initial learning rate used during training. It controls how much the model weights are adjusted based on the calculated gradients.
- **`fp16=True`**: This enables mixed-precision training using 16-bit floating-point numbers. This can significantly reduce memory usage and speed up training on compatible hardware (like the Tesla T4).
- **`logging_steps=10`**: This sets how often training progress (like loss) is logged during training. Here, it will log every 10 steps.
- **`save_steps=100`**: This determines how often the model checkpoints are saved during training. Checkpoints will be saved every 100 steps.
- **`save_total_limit=2`**: This limits the total number of checkpoints saved. Only the last 2 checkpoints will be kept.
- **`warmup_steps=50`**: This specifies the number of steps for a linear warmup phase of the learning rate. The learning rate will gradually increase from 0 to the initial learning rate (2e-4) over the first 50 steps.
- **`lr_scheduler_type="cosine"`**: This sets the type of learning rate scheduler used. The "cosine" scheduler will decrease the learning rate following a cosine curve after the warmup phase.
- **`report_to="none"`**: This disables reporting training metrics to external services like Weights & Biases (wandb).
- **`remove_unused_columns=False`**: This prevents the trainer from removing columns from the dataset that are not used by the model. This is important when using custom data processing functions.

In [12]:
print("Starting training...")
print("This may take several hours depending on your hardware.")

# Start training
trainer.train()

print("\nTraining completed!")

Starting training...
This may take several hours depending on your hardware.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1 | Total steps = 878
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,11.813800
20,7.067200
30,4.501700
40,3.232800
50,2.826900
60,2.585900
70,2.405600
80,2.198400
90,2.096400
100,2.061800



Training completed!


## Inference code to check how it does

In [14]:
# Prepare for inference
FastLanguageModel.for_inference(model)

# Test conversation
messages = [
    {
        "role": "user",
        "content": "is there a way to deal with unworthiness in personal life ?"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="medium"
).to("cuda")

# Generate response
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

print("Psychologist AI:")
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.9,
    top_p=0.99,
    streamer=text_streamer,
    do_sample=False,
)

Psychologist AI:
<|start|>assistant<|message|>Yes, there are ways to deal with feelings of unworthiness in your personal life. Here are a few suggestions that may help:

1. Practice self-compassion: Instead of beating yourself up for feeling unworthy, try to treat yourself with kindness and understanding. Remember that everyone makes mistakes and that it's okay to be imperfect.

2. Identify your strengths: Take some time to reflect on your strengths and accomplishments. Write them down and read them often to remind yourself of what you are good at.

3. Challenge negative thoughts: When you notice yourself thinking negative thoughts about yourself, try to challenge them. Ask yourself if there is evidence to support the thought, and if not, replace it with a more realistic and positive thought.

4.


## We will now save the model locally. We are saving the super quantized model of 4 bit

In [16]:
# Save merged 4-bit model locally
model.save_pretrained_merged(
    "oss_psychologist_merged_model_4bit",
    tokenizer,
    save_method = "merged_4bit_forced",
)

config.json: 0.00B [00:00, ?B/s]

Unsloth: Merging LoRA weights into 4bit model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['model.layers.0.mlp.router.linear', 'model.layers.1.mlp.router.linear', 'model.layers.2.mlp.router.linear', 'model.layers.3.mlp.router.linear', 'model.layers.4.mlp.router.linear', 'model.layers.5.mlp.router.linear', 'model.layers.6.mlp.router.linear', 'model.layers.7.mlp.router.linear', 'model.layers.8.mlp.router.linear', 'model.layers.9.mlp.router.linear', 'model.layers.10.mlp.router.linear', 'model.layers.11.mlp.router.linear', 'model.layers.12.mlp.router.linear', 'model.layers.13.mlp.router.linear', 'model.layers.14.mlp.router.linear', 'model.layers.15.mlp.router.linear', 'model.layers.16.mlp.router.linear', 'model.layers.17.mlp.router.linear', 'model.layers.18.mlp.router.linear', 'model.layers.19.mlp.router.linear', 'model.layers.20.mlp.router.linear', 'model.layers.21.mlp.router.linear', 'model.layers.22.mlp.router.linear', 'model.layers.23.mlp.router.linear', 'lm_head']. Updating config.
Unsloth: Saving merged 4bit model

In [18]:
# Then push to hub
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("merged_model_4bit")
model = AutoModelForCausalLM.from_pretrained(
    "oss_psychologist_merged_model_4bit",
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("merged_model_4bit")

OSError: merged_model_4bit is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
model.push_to_hub("your-username/oss_psychologist_merged_model_4bit", private=False)
tokenizer.push_to_hub("your-username/oss_psychologist_merged_model_4bit", private=False)

print("✅ Model successfully pushed to Hugging Face Hub!")

In [ ]:
# Push the model to Hugging Face Hub
model.push_to_hub("gbalachandhiran/gpt-oss-psychologist")
tokenizer.push_to_hub("gbalachandhiran/gpt-oss-psychologist") # Replace with your Hugging Face token or set it as a notebook secret

README.md:   0%|          | 0.00/604 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 21.4kB / 16.0MB            

Saved model to https://huggingface.co/gbalachandhiran/gpt-oss-psychologist


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ychologist/tokenizer.json:  90%|######### | 25.1MB / 27.9MB            

In [19]:
!dir

gpt-oss-psychologist		    sample_data
oss_psychologist_merged_model_4bit  unsloth_compiled_cache


In [3]:
# Load model directly
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="gbalachandhiran/gpt-oss-psychologist", # Replace with your username and model name
    max_seq_length=max_seq_length,
    dtype=None, # Auto detects Float16
    load_in_4bit=True, # Load in 4bit and not full
    # Add the suggested workaround for the ValueError
    attn_implementation="eager",
)

# You can now use the loaded model for inference
print("Model loaded successfully for inference.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: Unsloth: Please install unsloth_zoo via `pip install unsloth_zoo`

In [ ]:
# Prepare for inference
FastLanguageModel.for_inference(model)

# Test conversation
messages = [
    {
        "role": "user",
        "content": "How to deal with sleeplessness?"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="medium"
).to("cuda")

# Generate response
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

print("Psychologist AI:")
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.9,
    top_p=0.99,
    streamer=text_streamer,
    do_sample=False,
)

In [4]:
# Save the fine-tuned model and tokenizer
model.save_pretrained_merged("gbalachandhiran/gpt-oss-psychologist-merged", tokenizer, save_method = "merged_16bit",) # You can use "merged_4bit", "merged_8bit", "merged_16bit", "merged_bf16"
tokenizer.save_pretrained_pretrained("gbalachandhiran/gpt-oss-psychologist-merged")

NameError: name 'model' is not defined

In [2]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the source and destination paths
source_folder = "/content/oss_psychologist_merged_model_4bit"
destination_folder = "/content/drive/MyDrive/oss_psychologist_merged_model_4bit"

# Copy the folder to Google Drive
if os.path.exists(source_folder):
    shutil.copytree(source_folder, destination_folder)
    print(f"Folder '{source_folder}' successfully copied to '{destination_folder}'")
else:
    print(f"Source folder '{source_folder}' does not exist.")

Mounted at /content/drive
Folder '/content/oss_psychologist_merged_model_4bit' successfully copied to '/content/drive/MyDrive/oss_psychologist_merged_model_4bit'


After saving the model, you can push it to Hugging Face Hub. Replace `"your_username/gpt-oss-psychologist"` with your desired repository name.

## Inference code

In [4]:
from unsloth import FastLanguageModel
from huggingface_hub import login

## Loading the model from Hugging face

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "gbalachandhiran/oss_psychologist_merged_model_4bit",  # Path where you saved after training
    max_seq_length = 2048,  # Use the same as training
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.10.8: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## Loading model for Inference

In [7]:
FastLanguageModel.for_inference(model)  # Enable inference m

GptOssForCausalLM(
  (model): GptOssModel(
    (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
    (layers): ModuleList(
      (0-23): 24 x GptOssDecoderLayer(
        (self_attn): GptOssAttention(
          (q_proj): Linear4bit(in_features=2880, out_features=4096, bias=True)
          (k_proj): Linear4bit(in_features=2880, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=2880, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=4096, out_features=2880, bias=True)
        )
        (mlp): GptOssMLP(
          (router): GptOssTopKRouter(
            (linear): Linear(in_features=2880, out_features=32, bias=True)
          )
          (experts): GptOssExperts(
            (gate_up_projs): ModuleList(
              (0-31): 32 x Linear4bit(in_features=2880, out_features=5760, bias=True)
            )
            (down_projs): ModuleList(
              (0-31): 32 x Linear4bit(in_features=2880, out_features=2880, bias=True)
            )


In [8]:
# Test conversation
messages = [
    {
        "role": "user",
        "content": "How to deal with sleeplessness?"
    }
]


In [9]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="medium"
).to("cuda")

In [10]:
# Generate response
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

print("Psychologist AI:")
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.9,
    top_p=0.99,
    streamer=text_streamer,
    do_sample=False,
)

Psychologist AI:
<|start|>assistant<|channel|>commentary to=functions <|constrain|>json<|message|>{"id":"commentary","name":"functions","arguments":"{}"}<|call|><|start|>assistant<|channel|>commentary to=functions <|constrain|>json<|message|>{"id":"commentary","name":"functions","arguments":"{}"}<|call|><|start|>assistant<|channel|>final<|message|>Dealing with sleeplessness—whether it’s occasional insomnia, chronic sleep deprivation, or simply a restless night—can feel overwhelming. Below is a practical, step‑by‑step guide that blends science‑backed strategies with everyday habits. Feel free to adapt it to your own schedule, preferences, and health needs.

---

## 1. Diagnose the Root Cause

| Possible Cause | Quick Check |
|-----------------|--------------|
| **Stress /
